In [1]:
import pandas as pd
import numpy as np
import glob
from svv_read import Sub_SVV
import matplotlib.pyplot as plt

In [2]:
%matplotlib auto
pd.options.display.max_rows = None

Using matplotlib backend: MacOSX


In [3]:
# create data frame for results3
df = pd.DataFrame()

readfolder = '../DATA/EXP2/'
list_subject = glob.glob(readfolder+'/S*')
# list_subject

## check following outliers

 - check S50: -20º values (bucket moved during test)
 - check S45: very high angle and very short RT
 - check S46: similar S45
 - check S47: low angles

In [4]:
for sub in list_subject:
    SUB = Sub_SVV(sub)
    df = df.append(SUB.create_table(), ignore_index=True)


In [5]:
# snb.stripplot(x='freq', y='SVV', data = df)
# snb.stripplot(x='freq', y='SVV', data = df[(df['stim']=='SH') & (df['SVV']>2.5)])

## check for Subjective Visual Vesrtical over 2.5º in control condition

In [6]:
outliers = [(np.abs(df.loc[i,'SVV'])>2.5) and (df.loc[i,'stim']=='SH') for i in df.index]
bad_id = df[outliers].id.unique()
df.insert(len(df.columns), 'Outliers', df['id'].isin(bad_id))

## graphical representation of SVV data with outliers in red

In [7]:
bool_good_id = ~df['id'].isin(bad_id)
bool_SHDC = df['freq'] == 0  # select CTRL and DC stimulation

if False:    
    plt.figure()
    plt.subplot2grid((2,3),(0,0), rowspan=2)
    plt.plot(df[bool_SHDC].stim, df[bool_SHDC].SVV, 'o', label='All')
    plt.plot(df[bool_SHDC & ~bool_good_id].stim, df[bool_SHDC & ~bool_good_id].SVV, 'r.', ms=5, label='Outliers')
    plt.fill_between(['DC', 'SH'], [2.5, 2.5], -2.5, alpha=0.5, color='grey')
    plt.legend

    plt.title('Reference conditions')
    plt.xlabel('Stimulation')
    plt.ylabel('SVV (º)')

    plt.subplot2grid((2,3),(0,1), colspan=2)
    plt.plot(df[~bool_SHDC & (df['stim']=='AC')].freq, df[~bool_SHDC & (df['stim']=='AC')].SVV, 'o', alpha=0.4)
    plt.plot(df[~bool_SHDC & ~bool_good_id & (df['stim']=='AC')].freq, df[~bool_SHDC & ~bool_good_id & (df['stim']=='AC')].SVV, 'r.', ms=5)
    plt.fill_between([20, 160], [2.5, 2.5], -2.5, alpha=0.5, color='grey')

    plt.title('AC')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('SVV (º)')

    plt.subplot2grid((2,3),(1,1), colspan=2)
    plt.plot(df[~bool_SHDC & (df['stim']=='MF')].freq, df[~bool_SHDC & (df['stim']=='MF')].SVV, 'o', alpha=0.4)
    plt.plot(df[~bool_SHDC & ~bool_good_id & (df['stim']=='MF')].freq, df[~bool_SHDC & ~bool_good_id & (df['stim']=='MF')].SVV, 'r.', ms=5)
    plt.fill_between([20, 160], [2.5, 2.5], -2.5, alpha=0.5, color='grey')

    plt.title('MF')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('SVV (º)')

# Analysis over "good participants"

In [8]:
ndf = df[bool_good_id]  # remove subject with outliers SHAM data
ndf.head()

,id,trial,stim,freq,rep,Angle_init,SVV,Reaction_time,Outliers
0,S67,T01,MF,120.0,T1,-21.060645,-0.527312,5030.0,False
1,S67,T18,AC,60.0,T0,-20.586103,-0.519436,3316.0,False
2,S67,T12,DC,0.0,T1,-24.734157,-2.567490,4143.0,False
3,S67,T09,AC,160.0,T1,-21.389072,-0.389072,4357.0,False
4,S67,T20,MF,20.0,T1,-22.588575,-1.121908,6114.0,False


## create new table with delta-SVV as difference SVVcond - SVVctrl

In [ ]:
ctrl_svv = [ndf[(ndf['id']==i) & (ndf['stim']=='SH')].SVV.mean() for i in ndf.id.unique()]